In [6]:
import requests
import pandas as pd
import numpy as np
import bs4 as bs
import urllib.request
import lxml.html as lh

In [7]:
#Assigning URL
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [8]:
#Scarping the  URL using Beautiful Soup i.e Table Header and Table Data using HTML tags

def scrape_table_bs4(cname, cols):
    page  = urllib.request.urlopen(url).read()
    soup  = bs.BeautifulSoup(page, 'html.parser')
    table = soup.find("table", class_ = cname)
    header = [head.findAll(text = True)[0].strip() for head in table.find_all("th")]
    data   = [[td.findAll(text = True)[0].strip() for td in tr.find_all("td")]
    for tr in table.find_all("tr")]
    data    = [row for row in data if len(row) == cols]
    raw_df = pd.DataFrame(data, columns = header)
    return raw_df

In [9]:
def scrape_table_lxml(XPATH, cols):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    table_content = doc.xpath(XPATH)
    for table in table_content:
        headers = [th.text_content().strip() for th in table.xpath('//th')]
        headers = headers[0:3]
        data    = [[td.text_content().strip() for td in tr.xpath('td')] 
        for tr in table.xpath('//tbody/tr')]
        data    = [row for row in data if len(row) == cols]
        raw_df = pd.DataFrame(data, columns = headers)
        return raw_df


In [10]:
raw_data = scrape_table_bs4("wikitable", 3)


In [11]:
df = raw_data[~raw_data['Borough'].isin(['Not assigned'])]
df = df.sort_values(by=['Postcode','Borough','Neighbourhood'], ascending = [1,1,1]).reset_index(drop = True)
df.loc[df['Neighbourhood'] == 'Not assigned', ['Neighbourhood']] = df['Borough']
check_unassigned_sample = df.loc[df['Borough'] == 'Queen\'s Park']
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [12]:
#Display the first 5 rows
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
#Print the shape
df.shape

(103, 3)